In [1]:
import pandas as pd
import scipy.io
import re
import sys
import warnings
import pickle

In [2]:
from mvpa2.datasets.mri import fmri_dataset

Failed to import duecredit due to No module named 'duecredit'
/home/bsmith16/.conda/envs/py3_mvpa/lib/python3.8/site-packages/mvpa2/datasets/base.py:465: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def __init__(self, shape=None, sid=None, fid=None, dtype=np.float):


In [3]:
class BehavioralDataNotFoundForBrainDataException(Exception):
    """Behavioral data could not be matched to a subject."""
    pass

Replicating earlier work on mvpa. Try not to overly complicate it--the main point is just to verify we get similar results on a different package to validate prior work. ANd we are primarily interested in validating the very high cross-validation results I got with nltools. Should aim for readable code.

In [4]:
from mvpa2 import *

In [5]:
import sys

sys.path.append(os.path.abspath("../../ml/"))

from generic_utils import *

In [6]:


nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"

In [7]:
#brain_data_filepath = ml_data_folderpath + '/SST/Brain_Data_conditions_84subs_correct_cond.pkl'
#results_filepath=ml_data_folderpath + "/SST/mvpa_tt_res_v3_conditions_84subs_twoclasses_pfcmask.pkl"

In [8]:
include_exclude_list = pd.read_csv("../nsc_subject_exclusions.csv")

In [9]:
test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20211027T173724.csv")
test_train_df_raw = test_train_df_raw.merge(include_exclude_list[include_exclude_list.Task=='SST'],left_on='sub_label',right_on='SubjectId',how='left')
test_train_df_raw.loc[test_train_df_raw.Include.isna(),'Include'] = True
test_train_df = test_train_df_raw[test_train_df_raw.Include==True]
#exclude_subjects = ['DEV061','DEV185','DEV187','DEV189','DEV190','DEV192','DEV198','DEV203','DEV220','DEV221']
exclude_subjects = ['DEV098', 'DEV153', 'DEV154', 'DEV163', 'DEV170', 'DEV174', 'DEV181', 'DEV188', 'DEV200', 'DEV207', 'DEV208']
validation_exclude_subject = ['DEV098']
valiation_subjs = test_train_df.loc[test_train_df.SplitGroup=='Holdout','sub_label'].tolist()#only get the train subjects; ignore those previously marked hold-out

In [10]:
valiation_subjs_selected = [ts for ts in valiation_subjs if (ts not in exclude_subjects)]

In [11]:
individual_differences = pd.read_csv(ml_data_folderpath + "/data_by_ppt_2022_02_26.csv")
individual_differences = individual_differences.rename(columns={'SID':'subject'})
individual_differences['wave']=1

In [12]:
for kn in individual_differences.keys():
    print(kn +", " + str(type(individual_differences[kn])))
    print(individual_differences[kn].tolist())


subject, <class 'pandas.core.series.Series'>
['DEV001', 'DEV002', 'DEV004', 'DEV005', 'DEV006', 'DEV007', 'DEV008', 'DEV009', 'DEV010', 'DEV011', 'DEV012', 'DEV013', 'DEV014', 'DEV015', 'DEV016', 'DEV017', 'DEV018', 'DEV019', 'DEV020', 'DEV021', 'DEV022', 'DEV023', 'DEV024', 'DEV025', 'DEV026', 'DEV027', 'DEV028', 'DEV029', 'DEV030', 'DEV031', 'DEV032', 'DEV033', 'DEV034', 'DEV035', 'DEV036', 'DEV037', 'DEV038', 'DEV039', 'DEV040', 'DEV041', 'DEV042', 'DEV043', 'DEV044', 'DEV045', 'DEV046', 'DEV047', 'DEV048', 'DEV049', 'DEV050', 'DEV051', 'DEV052', 'DEV053', 'DEV054', 'DEV055', 'DEV056', 'DEV057', 'DEV058', 'DEV059', 'DEV060', 'DEV061', 'DEV062', 'DEV063', 'DEV064', 'DEV065', 'DEV066', 'DEV067', 'DEV068', 'DEV069', 'DEV070', 'DEV071', 'DEV072', 'DEV073', 'DEV074', 'DEV075', 'DEV076', 'DEV077', 'DEV078', 'DEV079', 'DEV080', 'DEV081', 'DEV082', 'DEV083', 'DEV084', 'DEV085', 'DEV086', 'DEV087', 'DEV088', 'DEV089', 'DEV090', 'DEV091', 'DEV093', 'DEV094', 'DEV095', 'DEV096', 'DEV097', 'DEV

In [13]:
import sys

sys.path.append(os.path.abspath("../../ml/"))

We probably actually want to start the pipeline from the betas rather than loading from pickle. to be continued...

In [14]:
from mvpa_pipeline_utils import get_Brain_Data_betas_as_mvpa_for_sub, import_beta_series_pymvpa2, sa_to_df

In [15]:
import mvpa2.datasets 

In [16]:
import nibabel as nib

In [17]:
from generic_utils import *

# CorrectGo/NoGo

## whole-brain

In [18]:
my_test_data =import_beta_series_pymvpa2(
    valiation_subjs_selected,'conditions',
    out_folder = ml_data_folderpath + "/SST/val_set_incomplete_",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_cond',# + mr['mask_name'],
    beta_processing_args={
        'spatially_concatenate': False,
        'mask': "beta"
#        'masking_threshold': mr['thresh']
    }
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV004
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV004/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV004/beta_0001.nii
 * Please note: warnings are printed only once, but underlying problem might occur many times *
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV004     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV004     1  
DEV007
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV007/SPM.mat
.
.
...verified th

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV065     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV065     1  
DEV066
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV066/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV066/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV066     1  
1  spm_spm:beta (0002) - Sn(1)

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV093     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV093     1  
DEV094
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV094/SPM.mat
not enough conditions for subject DEV094. Skipping this subject.
DEV095
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV095/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV095/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                 

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV113     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV113     1  
DEV114
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV114/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV114/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV114     1  
1  spm_spm:beta (0002) - Sn(1)

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV133     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV133     1  
DEV137
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV137/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV137/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV137     1  
1  spm_spm:beta (0002) - Sn(1)

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV172     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV172     1  
DEV176
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV176/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV176/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV176     1  
1  spm_spm:beta (0002) - Sn(1)

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV199     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV199     1  
DEV201
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV201/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV201/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV201     1  
1  spm_spm:beta (0002) - Sn(1)

## masks

## now all the other masks

In [19]:
mask_list=get_all_masks(ml_data_folderpath)

In [20]:
mask_list

,mask_name,mask_filepath,thresh
0,planning_association-test_z_FDR_0.01,/gpfs/projects/sanlab/shared/DEV/nonbids_data/...,0.1
1,executive function_association-test_z_FDR_0.01,/gpfs/projects/sanlab/shared/DEV/nonbids_data/...,0.1
2,response inhibition_association-test_z_FDR_0.01,/gpfs/projects/sanlab/shared/DEV/nonbids_data/...,0.1
3,inhibition_association-test_z_FDR_0.01,/gpfs/projects/sanlab/shared/DEV/nonbids_data/...,0.1
4,error_association-test_z_FDR_0.01,/gpfs/projects/sanlab/shared/DEV/nonbids_data/...,0.1
5,harvardoxford-cortical_prob_Frontal Orbital Co...,/gpfs/projects/sanlab/shared/DEV/nonbids_data/...,10.0
6,harvardoxford-cortical_prob_Inferior Frontal G...,/gpfs/projects/sanlab/shared/DEV/nonbids_data/...,10.0
7,harvardoxford-cortical_prob_Inferior Frontal G...,/gpfs/projects/sanlab/shared/DEV/nonbids_data/...,10.0


In [21]:
#mask_list = mask_list[mask_list.thresh==0.1]

In [22]:
import os.path
for v in valiation_subjs_selected:
    fmri_folder = nonbids_data_path + '/fMRI/fx/models/SST/wave1/conditions/'
    subject_folder = fmri_folder + 'sub-' + v + "/"
    spm_mat_filepath=subject_folder+"SPM.mat"
#    print(v)
    if (not os.path.exists(subject_folder)) or (not os.path.exists(spm_mat_filepath)):
        print(v, end="', '")
#        print("folder doesn't exist for subject " + v)
#    if :
#            print("spm.mat doesn't exist for subject " + v)


In [23]:
for i, mr in mask_list.iterrows():
    print(mr['mask_name'])
    my_test_data =import_beta_series_pymvpa2(
        valiation_subjs_selected,'conditions',
        out_folder = ml_data_folderpath + "/SST/val_set_incomplete_",
        conditions_to_include = ['CorrectGo','CorrectStop'],
        condition_count_required=2,
        supplementary_df = individual_differences,
        out_file_suffix = '_correct_cond_' + mr['mask_name'],
        beta_processing_args={
            'spatially_concatenate': False,
            'mask': mr['mask_filepath'],
            'masking_threshold': mr['thresh']
        }
    )

planning_association-test_z_FDR_0.01
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV004
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV004/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV004     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV004     1  
DEV007
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV007/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject 

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV078     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV078     1  
DEV080
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV080/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV080     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV080     1  
DEV081
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV081/SPM.mat
.
.
...verifi

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV110     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV110     1  
DEV111
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV111/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV111     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV111     1  
DEV112
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV112/SPM.mat
.
.
...verifi

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV137     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV137     1  
DEV138
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV138/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV138     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV138     1  
DEV144
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV144/SPM.mat
.
.
...verifi

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV182     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV182     1  
DEV183
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV183/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV183     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV183     1  
DEV191
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV191/SPM.mat
.
.
...verifi

...

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV070     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV070     1  
DEV072
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV072/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV072     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV072     1  
DEV075
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV075/SPM.mat
.
.
...verifi

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV103     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV103     1  
DEV106
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV106/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV106     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV106     1  
DEV108
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV108/SPM.mat
.
.
...verifi

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV127     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV127     1  
DEV131
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV131/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV131     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV131     1  
DEV133
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV133/SPM.mat
.
.
...verifi

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV178     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV178     1  
DEV179
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV179/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV179     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV179     1  
DEV180
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV180/SPM.mat
.
.
...verifi

.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV219     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV219     1  
DEV222
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV222/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV222     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV222     1  
DEV223
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx

In [24]:
from nilearn import plotting
plotting.plot_stat_map(masked_nifti.slicer[...,0])


NameError: name 'masked_nifti' is not defined

In [ ]:
plotting.plot_stat_map(first_dataset_img,vmax=18,cut_coords = (0,24,51))

In [25]:
my_test_data =import_beta_series_pymvpa2(
        valiation_subjs_selected[0:2],'conditions',
        out_folder = ml_data_folderpath + "/SST/val_set_incomplete_",
        conditions_to_include = ['CorrectGo','CorrectStop'],
        condition_count_required=2,
        supplementary_df = individual_differences,
        out_file_suffix = '_correct_cond_' + selected_mask['mask_name'],
        beta_processing_args={
            'spatially_concatenate': False,
            'mask': selected_mask['mask_filepath'],
            'masking_threshold': selected_mask['thresh']
        }
)

NameError: name 'selected_mask' is not defined

In [ ]:
plotting.plot_stat_map(nilearn.masking.unmask(my_test_data.samples[0],my_test_data.a['mask_nifti'].value))


## contrasted stop Minus Go

In [26]:
for i, mr in mask_list.iterrows():
    print(mr['mask_name'])

    my_test_data =import_beta_series_pymvpa2(
        valiation_subjs_selected,'conditions',
        out_folder = ml_data_folderpath + "/SST/val_set_incomplete_",
        conditions_to_include = ['CorrectGo','CorrectStop'],
        condition_count_required=2,
        supplementary_df = individual_differences,
        out_file_suffix = '_correct_stop_go_contrast_' + mr['mask_name'],
        beta_processing_args = {
            'contrast_condition_1':'CorrectStop',
            'contrast_condition_2':'CorrectGo',
            'spatially_concatenate':False,
            'mask': mr['mask_filepath'],
            'masking_threshold': mr['thresh']
        }
    )

planning_association-test_z_FDR_0.01
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV004
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV004/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV004     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV004     1  
DEV007
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV007/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject 

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV078     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV078     1  
DEV080
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV080/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV080     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV080     1  
DEV081
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV081/SPM.mat
.
.
...verifi

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV110     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV110     1  
DEV111
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV111/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV111     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV111     1  
DEV112
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV112/SPM.mat
.
.
...verifi

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV137     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV137     1  
DEV138
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV138/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV138     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV138     1  
DEV144
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV144/SPM.mat
.
.
...verifi

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV182     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV182     1  
DEV183
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV183/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV183     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV183     1  
DEV191
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV191/SPM.mat
.
.
...verifi

...

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV070     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV070     1  
DEV072
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV072/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV072     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV072     1  
DEV075
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV075/SPM.mat
.
.
...verifi

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV103     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV103     1  
DEV106
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV106/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV106     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV106     1  
DEV108
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV108/SPM.mat
.
.
...verifi

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV127     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV127     1  
DEV131
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV131/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV131     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV131     1  
DEV133
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV133/SPM.mat
.
.
...verifi

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV178     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV178     1  
DEV179
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV179/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV179     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV179     1  
DEV180
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV180/SPM.mat
.
.
...verifi

.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV219     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV219     1  
DEV222
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV222/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV222     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV222     1  
DEV223
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx

In [27]:

my_test_data =import_beta_series_pymvpa2(
    valiation_subjs_selected,'conditions',
    out_folder = ml_data_folderpath + "/SST/val_set_incomplete_",
    conditions_to_include = ['CorrectGo','CorrectStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_correct_stop_go_contrast',
    beta_processing_args = {
        'contrast_condition_1':'CorrectStop',
        'contrast_condition_2':'CorrectGo',
        'spatially_concatenate':False,
        'mask': 'beta'
    }
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/
DEV004
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV004/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV004/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV004     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV004     1  
DEV007
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV007/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series bet

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV066     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV066     1  
DEV070
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV070/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV070/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV070     1  
1  spm_spm:beta (0002) - Sn(1)

.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV095/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV095     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV095     1  
DEV096
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV096/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV096/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo 

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV114     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV114     1  
DEV116
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV116/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV116/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV116     1  
1  spm_spm:beta (0002) - Sn(1)

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV137     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV137     1  
DEV138
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV138/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV138/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV138     1  
1  spm_spm:beta (0002) - Sn(1)

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV176     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV176     1  
DEV178
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV178/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV178/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV178     1  
1  spm_spm:beta (0002) - Sn(1)

   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV201     1  
1  spm_spm:beta (0002) - Sn(1) CorrectStop*bf(1)  beta_0002.nii  DEV201     1  
DEV202
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV202/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/conditions/sub-DEV202/beta_0001.nii
   condition_index condition_label  \
0                0       CorrectGo   
1                1     CorrectStop   

                            raw_beta_description           beta subject  wave  
0    spm_spm:beta (0001) - Sn(1) CorrectGo*bf(1)  beta_0001.nii  DEV202     1  
1  spm_spm:beta (0002) - Sn(1)

# Post-error slowing

In [28]:
failure_mask_list=get_failure_masks(ml_data_folderpath)

## whole brain

In [29]:
my_bd = import_beta_series_pymvpa2(
    valiation_subjs_selected,'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/val_set_incomplete_",
    conditions_to_include = ['CorrectGoFollowingCorrectStop', 'CorrectGoFollowingFailedStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_post_stop_cg',
    beta_processing_args = {
        'mask':'beta'
    }
)

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/
DEV004
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV004/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV004/beta_0001.nii
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV004   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV004   

   wave  
0     1  
1     1  
DEV007
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV007/SPM.mat
.

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV063   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV063   

   wave  
0     1  
1     1  
DEV065
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV065/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV065/beta_0001.nii
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV090   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV090   

   wave  
0     1  
1     1  
DEV091
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV091/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV091/beta_0001.nii
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV110   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV110   

   wave  
0     1  
1     1  
DEV111
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV111/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV111/beta_0001.nii
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description

.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV122/beta_0001.nii
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV122   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV122   

   wave  
0     1  
1     1  
DEV123
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV123/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditio

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV156   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV156   

   wave  
0     1  
1     1  
DEV161
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV161/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV161/beta_0001.nii
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description

.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV183/beta_0001.nii
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV183   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV183   

   wave  
0     1  
1     1  
DEV191
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV191/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_condit

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV209   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV209   

   wave  
0     1  
1     1  
DEV211
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV211/SPM.mat
not enough conditions for subject DEV211. Skipping this subject.
DEV219
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV219/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV219/beta_0001.nii
   condition_index           

In [30]:
for i, mr in failure_mask_list.iterrows():
    print(mr['mask_name'])
    my_test_data =import_beta_series_pymvpa2(
        valiation_subjs_selected,'posterror_conditions',
        out_folder = ml_data_folderpath + "/SST/val_set_incomplete_",
        conditions_to_include = ['CorrectGoFollowingCorrectStop','CorrectGoFollowingFailedStop'],
        condition_count_required=2,
        supplementary_df = individual_differences,
        out_file_suffix = '_post_stop_cg_' + mr['mask_name'],
        beta_processing_args={
            'spatially_concatenate': False,
            'mask': mr['mask_filepath'],
            'masking_threshold': mr['thresh']
        }
    )

negative affect_association-test_z_FDR_0.01
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/
DEV004
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV004/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV004   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV004   

   wave  
0     1  
1     1  
DEV007
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV007/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  Corr

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV070   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV070   

   wave  
0     1  
1     1  
DEV072
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV072/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV072   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV072

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV100   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV100   

   wave  
0     1  
1     1  
DEV102
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV102/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV102   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV102

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV120   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV120   

   wave  
0     1  
1     1  
DEV121
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV121/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV121   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV121

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV161   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV161   

   wave  
0     1  
1     1  
DEV166
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV166/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV166   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV166

...

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV082   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV082   

   wave  
0     1  
1     1  
DEV088
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV088/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV088   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV088

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV111   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV111   

   wave  
0     1  
1     1  
DEV112
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV112/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV112   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV112

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV133   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV133   

   wave  
0     1  
1     1  
DEV137
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV137/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV137   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV137

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV178   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV178   

   wave  
0     1  
1     1  
DEV179
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV179/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV179   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV179

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV205   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV205   

   wave  
0     1  
1     1  
DEV209
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV209/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV209   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV209

## Contrast conditions 
### whole brain

In [31]:

import_beta_series_pymvpa2(
    valiation_subjs_selected,'posterror_conditions',
    out_folder = ml_data_folderpath + "/SST/val_set_incomplete_",
    conditions_to_include = ['CorrectGoFollowingCorrectStop', 'CorrectGoFollowingFailedStop'],
    condition_count_required=2,
    supplementary_df = individual_differences,
    out_file_suffix = '_post_stop_cg_contrast',
        beta_processing_args={
            'spatially_concatenate': False,
            'mask': 'beta',
            'contrast_condition_1':'CorrectGoFollowingCorrectStop',
            'contrast_condition_2':'CorrectGoFollowingFailedStop'
        }

)


/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/
DEV004
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV004/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV004/beta_0001.nii
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV004   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV004   

   wave  
0     1  
1     1  
DEV007
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV007/SPM.mat
.

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV063   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV063   

   wave  
0     1  
1     1  
DEV065
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV065/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV065/beta_0001.nii
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV090   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV090   

   wave  
0     1  
1     1  
DEV091
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV091/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV091/beta_0001.nii
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV110   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV110   

   wave  
0     1  
1     1  
DEV111
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV111/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV111/beta_0001.nii
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV122   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV122   

   wave  
0     1  
1     1  
DEV123
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV123/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV123/beta_0001.nii
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV156   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV156   

   wave  
0     1  
1     1  
DEV161
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV161/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV161/beta_0001.nii
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV183   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV183   

   wave  
0     1  
1     1  
DEV191
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV191/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV191/beta_0001.nii
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV209   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV209   

   wave  
0     1  
1     1  
DEV211
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV211/SPM.mat
not enough conditions for subject DEV211. Skipping this subject.
DEV219
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV219/SPM.mat
.
.
...verified that all expected betas exist! Processing...
setting the mask to the first image in the series beta_0001.nii
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV219/beta_0001.nii
   condition_index           

Dataset(array([[-0.67306054, -0.7049418 , -0.51332116, ...,  1.2334049 ,
         1.1313119 ,  1.7741904 ],
       [ 1.0860626 ,  0.5133235 ,  0.7907037 , ...,  0.55265284,
        -0.5912936 , -0.37530613],
       [ 0.07279411,  0.13117123,  0.5058233 , ..., -0.93452436,
        -0.29420507, -0.6585259 ],
       ...,
       [-0.2041617 , -0.0809966 , -0.08929375, ...,  0.26984993,
         0.23013747,  0.3215264 ],
       [-0.15895796, -0.39430094, -0.76438975, ...,  1.2640443 ,
         0.39267778,  0.7578813 ],
       [-1.486579  , -1.2231145 , -0.6871483 , ...,  0.62556916,
         0.22770607,  0.42524624]], dtype=float32), sa=SampleAttributesCollection(items=[ArrayCollectable(name='time_indices', doc=None, value=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

### masks

In [32]:
for i, mr in failure_mask_list.iterrows():
    print(mr['mask_name'])
    my_test_data =import_beta_series_pymvpa2(
        valiation_subjs_selected,'posterror_conditions',
        out_folder = ml_data_folderpath + "/SST/val_set_incomplete_",
        conditions_to_include = ['CorrectGoFollowingCorrectStop','CorrectGoFollowingFailedStop'],
        condition_count_required=2,
        supplementary_df = individual_differences,
        out_file_suffix = '_post_stop_cg_contrast_' + mr['mask_name'],
        beta_processing_args={
            'spatially_concatenate': False,
            'mask': mr['mask_filepath'],
            'masking_threshold': mr['thresh'],
            'contrast_condition_1':'CorrectGoFollowingCorrectStop',
            'contrast_condition_2':'CorrectGoFollowingFailedStop'
        }
    )

negative affect_association-test_z_FDR_0.01
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/
DEV004
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV004/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV004   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV004   

   wave  
0     1  
1     1  
DEV007
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV007/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  Corr

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV070   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV070   

   wave  
0     1  
1     1  
DEV072
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV072/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV072   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV072

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV100   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV100   

   wave  
0     1  
1     1  
DEV102
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV102/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV102   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV102

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV120   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV120   

   wave  
0     1  
1     1  
DEV121
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV121/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV121   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV121

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV161   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV161   

   wave  
0     1  
1     1  
DEV166
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV166/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV166   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV166

...

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV082   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV082   

   wave  
0     1  
1     1  
DEV088
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV088/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV088   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV088

.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV111   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV111   

   wave  
0     1  
1     1  
DEV112
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV112/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV112   
1  spm_spm:bet

.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV133   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV133   

   wave  
0     1  
1     1  
DEV137
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV137/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV137   
1  spm_spm:bet

.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV178   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV178   

   wave  
0     1  
1     1  
DEV179
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV179/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV179   
1  spm_spm:bet

   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV205   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV205   

   wave  
0     1  
1     1  
DEV209
/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/posterror_conditions/sub-DEV209/SPM.mat
.
.
...verified that all expected betas exist! Processing...
   condition_index                condition_label  \
0                0  CorrectGoFollowingCorrectStop   
1                1   CorrectGoFollowingFailedStop   

                                raw_beta_description           beta subject  \
0  spm_spm:beta (0001) - Sn(1) CorrectGoFollowing...  beta_0001.nii  DEV209   
1  spm_spm:beta (0002) - Sn(1) CorrectGoFollowing...  beta_0002.nii  DEV209